In [1]:
import Hartmannb

The result of custom_function at [0.95, 0.95, 0.95, 0.95, 0.95, 0.95] is 0.00011397265188861638


In [2]:
from botorch.test_functions.base import BaseTestProblem
from botorch.test_functions import Hartmann
import numpy as np 
import torch 
import random 
from botorch.models import SingleTaskGP
from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
# from botorch.fit import fit_gpytorch_model 
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

In [3]:
from botorch.acquisition.objective import ScalarizedPosteriorTransform

In [4]:
from scipy import stats
import matplotlib.pyplot as plt
from functools import partial
import torch.nn as nn
from botorch.models import MultiTaskGP
import scipy.stats
from botorch.acquisition import AcquisitionFunction
from botorch.acquisition import LogExpectedImprovement
import pandas as pd

In [5]:
import os

In [6]:
from botorch.acquisition import qLogNoisyExpectedImprovement

In [7]:
from botorch.acquisition import qLogExpectedImprovement


In [8]:
class NoisyShiftedAckley(BaseTestProblem):
    dim = 10   # 维度设置为10
    _bounds = [(-32.768, 32.768)] * dim  # Ackley 函数的输入范围

    def __init__(self, noise_std=0.1, shift=None, negate=False):
        super().__init__(negate=negate)
        self.noise_std = noise_std
        self.negate = negate  # 是否取负
        # 如果没有提供shift，就使用0向量
        if shift is not None:
            self.shift = torch.tensor(shift, dtype=torch.float32)
        else:
            self.shift = torch.zeros(self.dim, dtype=torch.float32)

    def evaluate_true(self, X):
        """评估真实的Ackley函数值"""
        # 确保shift和X在同一个设备上
        self.shift = self.shift.to(X.device)

        a = 20
        b = 0.2
        c = 2 * torch.pi
        n = X.shape[-1]

        # 打印维度信息
        print("X shape:", X.shape)
        print("Shift shape:", self.shift.shape)

        sum1 = torch.sum((X + self.shift) ** 2, dim=-1)
        sum2 = torch.sum(torch.cos(c * (X + self.shift)), dim=-1)
        term1 = -a * torch.exp(-b * torch.sqrt(sum1 / n))
        term2 = -torch.exp(sum2 / n)
        ackley_value = term1 + term2 + a + torch.exp(torch.tensor(1.0))
        return ackley_value

    def add_noise(self, Y):
        """在评估的Ackley函数值上添加噪声"""
        noise = torch.randn_like(Y) * self.noise_std
        return Y + noise

    def forward(self, X):
        """计算带噪声和shift的Ackley函数值，并根据需要取负"""
        true_values = self.evaluate_true(X)
        noisy_values = self.add_noise(true_values)

        # 如果需要取负，则返回负值
        if self.negate:
            return -noisy_values
        return noisy_values

In [9]:
ackley_fn1 = NoisyShiftedAckley(noise_std=0, shift=torch.tensor([0, 0, 0, 0, 0,0, 0, 0, 0, 0]), negate=True)
ackley_fn2 = NoisyShiftedAckley(noise_std=0, shift=torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,1]), negate=True)
ackley_fn3 = NoisyShiftedAckley(noise_std=0.7, shift=torch.tensor([0, 0, 0, 0, 0,0, 0, 0, 0, 0]), negate=True)

C:\Users\10925\AppData\Local\Temp\ipykernel_9640\3214432299.py:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [10]:
initial_points_task11 = [Hartmannb.generate_initial_samples_with_custom_function2(5,5, 10, -32.768, 32.768, seed=i)[0] for i in range(10)]
initial_points_task22 = None  # 如果你没有任务2的初始点，可以设置为 None
initial_points_task23 = [Hartmannb.generate_initial_samples_with_custom_function2(20,20, 10, -32.768, 32.768, seed=i)[0] for i in range(20)]

In [11]:
def fit_multi_task_gp_model_and_select_next_point(train_x, train_obj, device, task):
    input_dim = train_x.shape[1]
    outcome_dim = train_obj.shape[1]
    
    num_samples = train_x.size(0)
    task = task.expand(num_samples)  # 扩展 task 张量以匹配 train_x 的样本数

    # 将任务维度拼接到 train_x
    train_x_with_task = torch.cat([train_x, task.unsqueeze(-1)], dim=-1)

    # 修改 Normalize 变换的维度，以匹配拼接后的 train_x 维度
    input_transform = Normalize(d=train_x_with_task.shape[1])

    # 创建多任务模型，区分不同任务的训练数据
    model = MultiTaskGP(
        train_X=train_x_with_task,
        train_Y=train_obj,
        task_feature=train_x_with_task.shape[1] - 1,  # 任务特征索引
        input_transform=input_transform,  # 使用修改后的输入变换
        outcome_transform=Standardize(m=outcome_dim)
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()

    X_baseline = train_x[train_obj.flatten() == best_value.item()]

    qLogNEI = qLogNoisyExpectedImprovement(model=model, best_f=best_value, X_baseline=X_baseline)
    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogNEI,
        bounds=torch.tensor([[-32.768] * input_dim, [32.768] * input_dim], device=device, dtype=torch.float64),
        q=1,
        num_restarts=20,
        raw_samples=100,
        options={},
    )

    # 确保 new_point_analytic 是二维张量，形状为 (1, input_dim)
    new_point_analytic = new_point_analytic.unsqueeze(0)  # 确保形状为 (1, input_dim)

    return new_point_analytic


In [12]:
def fit_multi_task_gp_model_and_select_next_point1(train_x, train_obj, device, task):
    input_dim = train_x.shape[1]
    outcome_dim = train_obj.shape[1]
    
    num_samples = train_x.size(0)
    task = task.expand(num_samples)  # 扩展 task 张量以匹配 train_x 的样本数

    # 将任务维度拼接到 train_x
    train_x_with_task = torch.cat([train_x, task.unsqueeze(-1)], dim=-1)

    # 修改 Normalize 变换的维度，以匹配拼接后的 train_x 维度
    input_transform = Normalize(d=train_x_with_task.shape[1])

    # 创建和拟合多任务高斯过程模型
    model = MultiTaskGP(
        train_X=train_x_with_task,  # 使用拼接后的 train_x_with_task
        train_Y=train_obj,
        task_feature=-1,  # 假设最后一个维度是任务特征
        input_transform=input_transform,  # 使用匹配维度的 Normalize
        outcome_transform=Standardize(m=1),  # 假设输出为1维
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()

    # 使用 ScalarizedPosteriorTransform 来标量化多输出模型
    weights = torch.ones(1, device=device)  # 假设输出为1维
    posterior_transform = ScalarizedPosteriorTransform(weights=weights)

    # 使用 qLogExpectedImprovement 选择下一个采样点
    qLogEI = qLogExpectedImprovement(model=model, best_f=best_value, posterior_transform=posterior_transform)

    # 修改上下界为 Ackley 函数的上下界 [-32.768, 32.768]
    lower_bounds = [-32.768] * input_dim  # 根据输入的维度设置下界
    upper_bounds = [32.768] * input_dim   # 根据输入的维度设置上界
    
    bounds = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=torch.float64)

    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogEI,
        bounds=bounds,
        q=1,  # 采样一个点
        num_restarts=20,
        raw_samples=100,
        options={},
    )
    
    return new_point_analytic.squeeze(0)

    


In [13]:
results0 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task23,  # 传递生成的初始样本
    fit_task_fn=fit_multi_task_gp_model_and_select_next_point,
    device='cuda',
    task_type='multi'
)

X shape: torch.Size([5, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([20, 10])
Shift shape: torch.Size([10])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904, -27.3087, -26.2954,
          -9.9965,  22.7354,  17.1382],
        [ 28.9068, -18.2124,  16.7623,   8.2417,  -4.9984,  13.9407,   3.2422,
          32.0259,  -9.8914, -31.3283],
        [  1.5747,  31.3892, -29.8504, -23.8779, -18.2804,  25.3371, -14.1843,
           4.0460, -21.2126, -11.6977],
        [-28.8190, -10.7748,  13.4499,  21.2032,   8.2364, -15.9051,  23.9250,
         -21.9797,  11.4406,   5.4116],
        [-16.4862,  23.5604,  30.3572,  31.5481,   5.4023, -21.5482,  -1.7717,
           9.0194, -32.5698, -17.3561],
        [ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904, -27.3087, -26.2954,
          -9.9965,  22.7354,  17.1382],
        [ 28.9068, -18.2124,  16.7623,   8.2417,  -4.9984,  13.9407,   3.2422,
          32.0259,  -9.8914, -31.3283],
        [  1.5747,  31.3892, -29.8504, -23.8779,

C:\Users\10925\AppData\Roaming\Python\Python311\site-packages\linear_operator\utils\interpolation.py:71: UserWarning:

The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:80.)

C:\Users\10925\AppData\Roaming\Python\Python311\site-packages\linear_operator\utils\interpolation.py:71: UserWarning:

torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:623.)

F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch

X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
Nr. of optimization run: 0
New candidates are: tensor([[ 27.0976,  32.7680,  10.9414,   3.3336, -32.7680, -16.7048, -32.7680,
         -28.5110, -21.4726,  32.7680]], device='cuda:0', dtype=torch.float64)
Iteration 0 took 1.17 seconds


F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
Nr. of optimization run: 1
New candidates are: tensor([[ 32.5147,  32.7680,  32.7680,  32.7680,  32.7680,  32.7680,   8.8440,
         -32.7680,  32.7680,  -3.8009]], device='cuda:0', dtype=torch.float64)
Iteration 1 took 0.62 seconds


...

F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
Nr. of optimization run: 22
New candidates are: tensor([[ -3.0931,   5.0459,   8.5574,   0.4945,  30.1870,   2.1726, -27.1300,
          21.3621,   2.2905, -20.4174]], device='cuda:0', dtype=torch.float64)
Iteration 22 took 1.28 seconds


F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
Nr. of optimization run: 23
New candidates are: tensor([[-28.9626, -15.8210,   8.2610,  -9.4857,  29.1102,   9.6233, -26.2846,
          15.1452,   2.0212,  14.9940]], device='cuda:0', dtype=torch.float64)
Iteration 23 took 0.92 seconds
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
Nr. of optimization run: 24
New candidates are: tensor([[-29.5471, -14.9040,  12.3827,   9.6877,  30.6205,  11.2873, -22.9025,
          15.5454,  27.7574,   1.2358]], device='cuda:0', dtype=torch.float64)
Iteration 24 took 0.92 seconds
最优点: [-21.1843106603998, -21.1843106603998, -21.054468281129733, -21.054468281129733, -20.791562956324817, -20.791562956324817, -20.791562956324817, -20.791562956324817, -20.791562956324817, -20.791562956324817, -20.404230066302368, -20.404230066302368, -20.404230066302368, -20.40423006630

F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



In [14]:
def fit_gp_model_and_select_next_point(train_x, train_obj, device):
    # 创建和拟合高斯过程模型
    model = SingleTaskGP(
        train_X=train_x,
        train_Y=train_obj,
        input_transform=Normalize(d=train_x.shape[1]),  # 使用输入的维度
        outcome_transform=Standardize(m=1),  # 输出为一维
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()


    X_baseline = train_x[train_obj.flatten() == best_value.item()]

    qLogNEI = qLogNoisyExpectedImprovement(model=model, best_f=best_value, X_baseline=X_baseline)
    # 修改上下界为 Ackley 函数的上下界 [-32.768, 32.768]
    lower_bounds = [-32.768] * train_x.shape[1]  # 根据输入的维度设置下界
    upper_bounds = [32.768] * train_x.shape[1]   # 根据输入的维度设置上界
    
    bounds = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=torch.float64)
    
    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogNEI,
        bounds=bounds,
        q=1,  # 采样一个点
        num_restarts=20,
        raw_samples=100,
        options={},
    )
    
    return new_point_analytic.squeeze(0)


In [15]:
def fit_gp_model_and_select_next_point1(train_x, train_obj, device):
    # 创建和拟合高斯过程模型
    model = SingleTaskGP(
        train_X=train_x,
        train_Y=train_obj,
        input_transform=Normalize(d=train_x.shape[1]),  # 使用输入的维度
        outcome_transform=Standardize(m=1),  # 输出为一维
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()



    qLogEI = qLogExpectedImprovement(model=model, best_f=best_value)

    # 修改上下界为 Ackley 函数的上下界 [-32.768, 32.768]
    lower_bounds = [-32.768] * train_x.shape[1]  # 根据输入的维度设置下界
    upper_bounds = [32.768] * train_x.shape[1]   # 根据输入的维度设置上界
    
    bounds = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=torch.float64)
    
    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogEI,
        bounds=bounds,
        q=1,  # 采样一个点
        num_restarts=20,
        raw_samples=100,
        options={},
    )
    
    return new_point_analytic.squeeze(0)

In [16]:
results1 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task22,  # 传递生成的初始样本
    fit_task_fn=fit_gp_model_and_select_next_point,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)


X shape: torch.Size([5, 10])
Shift shape: torch.Size([10])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904, -27.3087, -26.2954,
          -9.9965,  22.7354,  17.1382],
        [ 28.9068, -18.2124,  16.7623,   8.2417,  -4.9984,  13.9407,   3.2422,
          32.0259,  -9.8914, -31.3283],
        [  1.5747,  31.3892, -29.8504, -23.8779, -18.2804,  25.3371, -14.1843,
           4.0460, -21.2126, -11.6977],
        [-28.8190, -10.7748,  13.4499,  21.2032,   8.2364, -15.9051,  23.9250,
         -21.9797,  11.4406,   5.4116],
        [-16.4862,  23.5604,  30.3572,  31.5481,   5.4023, -21.5482,  -1.7717,
           9.0194, -32.5698, -17.3561]], device='cuda:0', dtype=torch.float64)
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: tor

In [17]:
def fit_gp_model_and_select_next_point2(train_x, train_obj, device):
        # 生成样本点
    initial_samples_np = Hartmannb.generate_initial_samples(num_samples=1, dim=10, lower_bound=-32.768, upper_bound=32.768)
    
    # 转换为张量
    initial_samples_tensor = torch.tensor(initial_samples_np, dtype=torch.float64)
    return initial_samples_tensor.squeeze(0)

In [18]:
results2 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task22,  # 传递生成的初始样本
    fit_task_fn=fit_gp_model_and_select_next_point2,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)

X shape: torch.Size([5, 10])
Shift shape: torch.Size([10])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904, -27.3087, -26.2954,
          -9.9965,  22.7354,  17.1382],
        [ 28.9068, -18.2124,  16.7623,   8.2417,  -4.9984,  13.9407,   3.2422,
          32.0259,  -9.8914, -31.3283],
        [  1.5747,  31.3892, -29.8504, -23.8779, -18.2804,  25.3371, -14.1843,
           4.0460, -21.2126, -11.6977],
        [-28.8190, -10.7748,  13.4499,  21.2032,   8.2364, -15.9051,  23.9250,
         -21.9797,  11.4406,   5.4116],
        [-16.4862,  23.5604,  30.3572,  31.5481,   5.4023, -21.5482,  -1.7717,
           9.0194, -32.5698, -17.3561]], device='cuda:0', dtype=torch.float64)
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: tor

C:\Users\10925\AppData\Local\Temp\ipykernel_9640\1636914412.py:6: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Nr. of optimization run: 9
New candidates are: tensor([[ 16.1848, -22.6071,  21.7495,  17.6619,  19.7384,   9.3569,  10.5354,
         -15.2757,  -5.9545, -17.2870]], device='cuda:0', dtype=torch.float64)
Iteration 9 took 0.00 seconds
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
Nr. of optimization run: 10
New candidates are: tensor([[ 16.0616, -22.9069, -17.5519,  11.2560, -13.9732, -20.3024,  -8.2516,
          29.4474,   6.9074,   3.3936]], device='cuda:0', dtype=torch.float64)
Iteration 10 took 0.02 seconds
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
Nr. of optimization run: 11
New candidates are: tensor([[-30.6735,  14.5396,  20.8929,   8.5248, -18.7606,  -9.1674, -12.1528,
         -16.9763, -14.9053, -26.7266]], device='cuda:0', dtype=torch.float64)
Iteration 11 took 0.00 seconds
X shape: torch.Size([1, 10])
Shift shape: torch.Size([1

In [19]:
from pyDOE2 import lhs
from scipy.stats import qmc

In [20]:
# 生成 LHS 样本并缩放到指定上下界
def generate_lhs_samples(num_features, num_samples, lower_bounds, upper_bounds):
    # 生成 LHS 样本
    lhs_sampler = qmc.LatinHypercube(d=num_features)
    samples = lhs_sampler.random(n=num_samples)
    
    # 缩放样本到指定的上下界
    scaled_samples = qmc.scale(samples, lower_bounds, upper_bounds)
    
    return scaled_samples


# 管理 LHS 样本并按顺序检索它们的类
class SampleProvider:
    def __init__(self, num_features, num_samples):
        self.samples_np = generate_lhs_samples(num_features, num_samples, lower_bounds, upper_bounds)
        self.samples_tensor = torch.tensor(self.samples_np, dtype=torch.float64)
        self.current_index = 0

    def get_next_sample(self):
        if self.current_index < self.samples_tensor.size(0):
            sample = self.samples_tensor[self.current_index]
            self.current_index += 1
            return sample
        else:
            raise IndexError("没有更多样本可用了。")


In [21]:
initial_points_task1 = [Hartmannb.generate_initial_samples_with_custom_function(5, 20, 10, -32.768, 32.768, seed=i)[0] for i in range(10)]
initial_points_task2 = None  # 如果你没有任务2的初始点，可以设置为 None

In [22]:
# 示例使用
num_features = 10
num_samples = 2500
lower_bounds = [-32.768] * 10
upper_bounds = [32.768] * 10
sample_provider = SampleProvider(num_features, num_samples)



# 检索一个样本（每次调用函数时都会这样做）
next_sample = fit_gp_model_and_select_next_point2(None, None, None)
print(next_sample)

tensor([  8.5811,  25.4564, -32.1177,   2.9919,   8.7320,  17.8392,  22.6491,
         -6.9947, -20.2900,  -4.3410], device='cuda:0', dtype=torch.float64)


C:\Users\10925\AppData\Local\Temp\ipykernel_9640\1636914412.py:6: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [23]:
# 类似你提供的函数
def zj(train_x, train_obj, device):
    return sample_provider.get_next_sample()

In [24]:
results3 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    fit_task_fn=zj,  # 你的模型拟合函数
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task1,  # 传递生成的初始样本
    initial_points_task2=initial_points_task2,  # 传递生成的初始样本
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)





X shape: torch.Size([5, 10])
Shift shape: torch.Size([10])
tensor([[-22.9044, -22.2731,  24.8236,  10.7274,  21.6020,  19.0823,   5.0997,
         -20.6372,  11.5213,  11.4049],
        [-15.1732,  27.0148,  11.6194,  -8.9067,  18.7185,  24.0373,  19.5741,
         -11.3776, -29.9506,  -5.5990],
        [  5.3413,  -1.5632,  -5.7718,  21.8122,  -5.0768, -29.4799, -30.0023,
          -2.5873,  -5.0487, -10.2613],
        [ 16.5186,  -7.2357, -16.8016,  -1.6240, -27.9076, -15.3234,  31.1394,
          24.0685,  26.7653, -20.3820],
        [ 31.0975,  13.6636, -30.8306, -22.7288, -10.2823,   0.9630,  -7.8269,
          10.4519, -13.8612,  30.9141]], device='cuda:0')
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shif

In [25]:
results4 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task23,  # 传递生成的初始样本
    fit_task_fn=fit_multi_task_gp_model_and_select_next_point1,
    device='cuda',
    task_type='multi'
)

X shape: torch.Size([5, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([20, 10])
Shift shape: torch.Size([10])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904, -27.3087, -26.2954,
          -9.9965,  22.7354,  17.1382],
        [ 28.9068, -18.2124,  16.7623,   8.2417,  -4.9984,  13.9407,   3.2422,
          32.0259,  -9.8914, -31.3283],
        [  1.5747,  31.3892, -29.8504, -23.8779, -18.2804,  25.3371, -14.1843,
           4.0460, -21.2126, -11.6977],
        [-28.8190, -10.7748,  13.4499,  21.2032,   8.2364, -15.9051,  23.9250,
         -21.9797,  11.4406,   5.4116],
        [-16.4862,  23.5604,  30.3572,  31.5481,   5.4023, -21.5482,  -1.7717,
           9.0194, -32.5698, -17.3561],
        [ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904, -27.3087, -26.2954,
          -9.9965,  22.7354,  17.1382],
        [ 28.9068, -18.2124,  16.7623,   8.2417,  -4.9984,  13.9407,   3.2422,
          32.0259,  -9.8914, -31.3283],
        [  1.5747,  31.3892, -29.8504, -23.8779,

In [26]:
results5 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task22,  # 传递生成的初始样本
    fit_task_fn=fit_gp_model_and_select_next_point1,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)


X shape: torch.Size([5, 10])
Shift shape: torch.Size([10])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904, -27.3087, -26.2954,
          -9.9965,  22.7354,  17.1382],
        [ 28.9068, -18.2124,  16.7623,   8.2417,  -4.9984,  13.9407,   3.2422,
          32.0259,  -9.8914, -31.3283],
        [  1.5747,  31.3892, -29.8504, -23.8779, -18.2804,  25.3371, -14.1843,
           4.0460, -21.2126, -11.6977],
        [-28.8190, -10.7748,  13.4499,  21.2032,   8.2364, -15.9051,  23.9250,
         -21.9797,  11.4406,   5.4116],
        [-16.4862,  23.5604,  30.3572,  31.5481,   5.4023, -21.5482,  -1.7717,
           9.0194, -32.5698, -17.3561]], device='cuda:0', dtype=torch.float64)
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: torch.Size([1, 10])
Shift shape: torch.Size([10])
X shape: tor

In [27]:
results_list = [results0,results1, results2, results3, results4, results5]
results_names = ['results0','results1', 'results2', 'results3' ,'results4', 'results5']

# 初始化一个空字典来存储展平后的数据
flattened_results = {}

# 遍历所有结果数组
for i, results in enumerate(results_list):
    if isinstance(results, torch.Tensor):
        results_np = results.cpu().numpy()  # 转为 numpy 数组
    else:
        results_np = results  # 如果已经是 numpy 数组，直接使用
    
    # 获取数组的维度
    shape_len = len(results_np.shape)
    
    if shape_len == 3:
        # 3D 数组的形状 (num_experiments, num_samples, num_metrics)
        num_experiments, num_samples, num_metrics = results_np.shape
        # 展平为 2D 数组
        flattened_array = results_np.reshape(num_experiments * num_samples, num_metrics)
    elif shape_len == 2:
        # 2D 数组的形状 (num_experiments, num_metrics)
        flattened_array = results_np
        num_experiments, num_metrics = flattened_array.shape
    else:
        raise ValueError(f"Unexpected number of dimensions: {shape_len}")
    
    # 将每个数组的列命名为 'results1_Metric_1', 'results1_Metric_2', ...
    columns = [f"{results_names[i]}_Metric_{j+1}" for j in range(num_metrics)]
    
    # 将展平后的数组存入字典，方便后面转换为 DataFrame
    flattened_results.update({col: flattened_array[:, j] for j, col in enumerate(columns)})

# 创建 DataFrame
df_results = pd.DataFrame(flattened_results)

# 文件名
csv_filename = 'all_results_ackley10.噪声2.csv'

# 检查 CSV 文件是否已经存在
if os.path.exists(csv_filename):
    # 如果文件存在，先读取现有文件
    df_existing = pd.read_csv(csv_filename)
    # 合并新结果到现有的 DataFrame 中
    df_results = pd.concat([df_existing, df_results], ignore_index=True)

# 保存或追加为 CSV 文件
df_results.to_csv(csv_filename, index=False)

print(f"所有结果已保存到 {csv_filename} 文件中。")

所有结果已保存到 all_results_ackley10.噪声2.csv 文件中。
